RUN SERVER IN KAGGLE. GPU: T4x2

In [1]:
!conda create -n myenv python=3.10 -y
!conda activate myenv

# Install vLLM with CUDA 12.1.
!pip install vllm

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



## Package Plan ##

  environment location: /opt/conda/envs/myenv

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    bzip2-1.0.8                |       hd590300_5         248 KB  conda-forge
    ld_impl_linux-64-2.40      |       h55db66e_0         697 KB  conda-forge
    libgcc-ng-13.2.0           |       h77fa898_7         758 KB  conda-forge
    libgomp-13.2.0             |       h77fa898_7         412 KB  conda-forg

In [2]:
!pip -qq install -U xformers 
!pip -qq install langchain optimum qdrant-client wikipedia FastAPI uvicorn pyngrok

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 5.26.1 which is incompatible.
apache-

In [3]:
GENERATE_MODEL_NAME="brown1808/vietzephyr-7b-lora-8bit"
EMBEDDINGS_MODEL_NAME="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
QDRANT_URL = "<ANT_URL>"
QDRANT_COLLECTION_NAME = "<QDRANT_COLLECTION_NAME>"
NGROK_STATIC_DOMAIN = "<NGROK_STATIC_DOMAIN>"
NGROK_TOKEN=          "<NGROK_TOKEN>"
HUGGINGFACE_API_KEY = "<HUGGINGFACE_API_KEY>"
QDRANT_API_KEY =      "<QDRANT_API_KEY>"

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from optimum.bettertransformer import BetterTransformer
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_rerank = AutoModelForSequenceClassification.from_pretrained('amberoad/bert-multilingual-passage-reranking-msmarco').to(device)
model_rerank = BetterTransformer.transform(model_rerank)
tokenizer_rerank = AutoTokenizer.from_pretrained('amberoad/bert-multilingual-passage-reranking-msmarco')

2024-05-15 06:44:22.349808: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 06:44:22.349933: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 06:44:22.515110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
from langchain.schema.document import Document
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.retrievers import WikipediaRetriever
from typing import List
class RerankRetriever(VectorStoreRetriever):
    vectorstore: VectorStoreRetriever
    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query]*len(candidates)
        features = tokenizer_rerank(queries, candidates, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
            # relevant_docs = docs[indices[0]]
        return [docs[indices[0]],docs[indices[1]]]
class RerankWikiRetriever(VectorStoreRetriever):
    vectorstore: WikipediaRetriever
    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query]*len(candidates)
        features = tokenizer_rerank(queries, candidates,  padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
            # relevant_docs = docs[indices[0]]
        return [docs[indices[0]],docs[indices[1]]]

In [6]:
from langchain.retrievers import WikipediaRetriever
from langchain.vectorstores import Qdrant
from langchain.llms import HuggingFacePipeline
from qdrant_client import QdrantClient
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.chains import RetrievalQA,MultiRetrievalQAChain
from langchain.llms import VLLM
from langchain.llms import HuggingFaceHub

class LLMServe:
    def __init__(self) -> None:
      self.embeddings = self.load_embeddings()
      self.current_source = "wiki"
      self.retriever = self.load_retriever(retriever_name = self.current_source,embeddings=self.embeddings)
      self.pipe = self.load_model_pipeline(max_new_tokens=150)
      self.prompt = self.load_prompt_template()
      self.rag_pipeline = self.load_rag_pipeline(llm=self.pipe,
                                            retriever=self.retriever,
                                            prompt=self.prompt)
    def load_embeddings(self):
      embeddings = HuggingFaceInferenceAPIEmbeddings(
          model_name=EMBEDDINGS_MODEL_NAME,
          api_key = HUGGINGFACE_API_KEY,
          model_kwargs = {'device': "auto"}
      )
      return embeddings

    def load_retriever(self,retriever_name,embeddings):
      retriever=None
      if retriever_name == "wiki":
        retriever = RerankWikiRetriever(vectorstore = WikipediaRetriever(lang="vi",
                                       doc_content_chars_max=300,top_k_results=5))
      else:
        client = QdrantClient(
            url=QDRANT_URL,api_key=QDRANT_API_KEY, prefer_grpc=False
        )
        db = Qdrant(client=client,
                    embeddings=embeddings,
                    collection_name=QDRANT_COLLECTION_NAME)

        retriever = RerankRetriever(vectorstore = db.as_retriever(search_kwargs={"k":5}))

      return retriever

    def load_model_pipeline(self,max_new_tokens=150):
      llm = VLLM(
          model=GENERATE_MODEL_NAME,
          trust_remote_code=True,  # mandatory for hf models
          max_new_tokens=max_new_tokens,
            # temperature=1.0,
            # top_k=50,
            # top_p=0.9,
          top_k=5,
          top_p=0.95,
          temperature=0.1,
          dtype="half",
#           vllm_kwargs={"quantization": "awq"}

      )
      return llm

    def load_prompt_template(self):
      query_template = "<|im_start|>systemBạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác nhất, nếu thông tin không liên quan đến câu hỏi thì trả lời không biết\n{context}<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant"
      prompt = PromptTemplate(template=query_template,
                        input_variables= ["context","question"])
      return prompt

    def load_rag_pipeline(self,llm,retriever,prompt):
      rag_pipeline = RetrievalQA.from_chain_type(
      llm=llm, chain_type='stuff',
      retriever=retriever,
      chain_type_kwargs={
      "prompt": prompt
      },
      return_source_documents=True)
      return rag_pipeline

    def rag(self,source):
      if source == self.current_source:
        return self.rag_pipeline
      else:
        self.retriever = self.load_retriever(retriever_name=source,embeddings=self.embeddings)
        self.rag_pipeline = self.load_rag_pipeline(llm=self.pipe,
                                      retriever=self.retriever,
                                      prompt=self.prompt)
        self.current_source = source
        return self.rag_pipeline

In [7]:
!pip install -e .

Obtaining file:///kaggle/working
ERROR: file:///kaggle/working does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [9]:
CUDA_LAUNCH_BLOCKING=1,2

In [10]:
app = LLMServe()

2024-05-15 06:45:05,315	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

INFO 05-15 06:45:05 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='brown1808/vinallama-2b-custom-ver2', speculative_config=None, tokenizer='brown1808/vinallama-2b-custom-ver2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=brown1808/vinallama-2b-custom-ver2)


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

INFO 05-15 06:45:06 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-15 06:45:06 selector.py:69] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 05-15 06:45:06 selector.py:32] Using XFormers backend.
INFO 05-15 06:45:07 weight_utils.py:199] Using model weights format ['*.safetensors']


model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/554M [00:00<?, ?B/s]

INFO 05-15 06:45:28 model_runner.py:175] Loading model weights took 5.2088 GB
INFO 05-15 06:45:30 gpu_executor.py:114] # GPU blocks: 1471, # CPU blocks: 819
INFO 05-15 06:45:32 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-15 06:45:32 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 05-15 06:45:48 model_runner.py:1017] Graph capturing finished in 16 secs.


In [12]:
from typing import Union
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder
from fastapi import FastAPI
origins = ["*"]
app_api = FastAPI()
app_api.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app_api.get("/")
def read_root():
    return "API RAG"

@app_api.get("/rag/{source}")
async def read_item(source: str, q: str | None = None):
    if q:
        data = app.rag(source=source)(q)
        sources = []
        for docs in data["source_documents"]:
            sources.append(docs.to_json()["kwargs"])
        res = {
            "result" : data["result"],
            "source_documents":sources
        }
        return JSONResponse(content=jsonable_encoder(res))
    return None


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
ngrok.set_auth_token(NGROK_TOKEN)
ngrok_tunnel = ngrok.connect(8000,domain=NGROK_STATIC_DOMAIN)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app_api, port=8000)

INFO:     Started server process [34]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://ruling-plainly-jaguar.ngrok-free.app
INFO:     14.176.232.239:0 - "OPTIONS /rag/vi-medical?q=Khi%20n%C3%A0o%20VNVC%20m%E1%BB%9Bi%20c%C3%B3%20d%E1%BB%8Bch%20v%E1%BB%A5%20ti%C3%AAm%20vaccine%20Covid-19?%20Gi%C3%A1%20kho%E1%BA%A3ng%20bao%20nhi%C3%AAu%20m%E1%BB%99t%20m%C5%A9i%20cho%20t%E1%BB%ABng%20lo%E1%BA%A1i%20AstraZeneca,%20Pfizer,%20v%C3%A0%20Moderna%20%E1%BA%A1? HTTP/1.1" 200 OK


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/opt/conda/lib/python3.10/site-packages/optimum/bettertransformer/models/encoder_models.py:301: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, ~attention_mask)


INFO:     14.176.232.239:0 - "GET /rag/vi-medical?q=Khi%20n%C3%A0o%20VNVC%20m%E1%BB%9Bi%20c%C3%B3%20d%E1%BB%8Bch%20v%E1%BB%A5%20ti%C3%AAm%20vaccine%20Covid-19?%20Gi%C3%A1%20kho%E1%BA%A3ng%20bao%20nhi%C3%AAu%20m%E1%BB%99t%20m%C5%A9i%20cho%20t%E1%BB%ABng%20lo%E1%BA%A1i%20AstraZeneca,%20Pfizer,%20v%C3%A0%20Moderna%20%E1%BA%A1? HTTP/1.1" 200 OK


INFO:     14.176.232.239:0 - "OPTIONS /rag/vi-medical?q=Ph%C3%B2ng%20tr%C3%A1nh%20gan%20nhi%E1%BB%85m%20m%E1%BB%A1 HTTP/1.1" 200 OK


Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.35s/it]

INFO:     14.176.232.239:0 - "GET /rag/vi-medical?q=Ph%C3%B2ng%20tr%C3%A1nh%20gan%20nhi%E1%BB%85m%20m%E1%BB%A1 HTTP/1.1" 200 OK
